# Reducing

Reducing is a principles that will be used to merge two different states, when both are supposed to be used in the following nodes.

In [3]:
from typing import TypedDict, Annotated
from langgraph.graph import START, END, StateGraph

### Sequential

Note that if you've defined the reducer for the state attribute of the graph, the sequential state updates will also be merged.

---

Consider the following example: the `attr1` does not have a reducer function, but to the `attr2` does. Sequentially connected nodes simply return state updates with a list with a single random value.

In [4]:
import random
import operator


class State(TypedDict):
    attr1: list[int]
    attr2: Annotated[list[int], operator.add]


def gen_value():
    return random.randint(5, 10)


def node1(state: State) -> dict:
    return {"attr1": [gen_value()], "attr2": [gen_value()]}


def node2(state: State) -> State:
    return State(attr1=[gen_value()], attr2=[gen_value()])


graph = (
    StateGraph(State)
    .add_node("node1", node1)
    .add_node("node2", node2)

    .add_edge(START, "node1")
    .add_edge("node1", "node2")
    .add_edge("node2", END)
    .compile()
)

The following cell applies the graph.

In [42]:
graph.invoke(State(attr1=[1], attr2=[2]))

{'attr1': [9], 'attr2': [2, 5, 10]}

Despite the fact that there is no branching in the graph, the generated values are **appended** to attributes that have reducers, rather than replacing them. For attributes without reducers, `InvalidUpdateError` is not raised because there is no **ambiguous** update - the new value just replaces the previous value in the graph.

### Messages

In agent flow development, it is common practice to merge the messages from the different nodes. To add new messages to the list of messages, use `langcahin.graph.message.add_messages` as a reduce function.

---

The following cell generates the graph using the `langgraph.graph.message.add_message` as a reducer function. The only node that the graph uses simply adds an extra message to the messages list.

In [ ]:
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, BaseMessage


class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]


def a(state: State) -> State:
    return State(messages=[HumanMessage("Message from the a")])


graph = (
    StateGraph(State)
    .add_node("a", a)
    .add_edge("__start__", "a")
    .add_edge("a", "__end__")
    .compile()
)

The following cell shows how an additional message from node "a" is added to the input message.

In [22]:
graph.invoke(State(messages=[HumanMessage("Input message")]))["messages"]

[HumanMessage(content='Input message', additional_kwargs={}, response_metadata={}, id='373072fe-498a-45c2-8979-d7765e1c3392'),
 HumanMessage(content='Message from the a', additional_kwargs={}, response_metadata={}, id='41f3346a-af40-493a-b854-01df32747e7d')]